#### Create a model with the duplicated regressor

In [11]:
import tensorflow as tf
from dlomix.losses import masked_spectral_distance, masked_pearson_correlation_distance
import keras.backend as K
from dlomix.data import load_processed_dataset
import os

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = ''

In [3]:
model = tf.keras.models.load_model("/cmnfs/proj/bmpc_dlomix/models/baseline_models/noptm_baseline_full_bs1024_naivemods/d961f940-d142-4102-9775-c1f8b4373c91.keras")
model.summary()

/nfs/home/students/l.willruth/miniconda3/envs/dlomix2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Avaliable feature extractors are (use the key of the following dict and pass it to features_to_extract in the Dataset Class):
{
   "atom_count": "Atom count of PTM.",
   "delta_mass": "Delta mass of PTM.",
   "mod_gain": "Gain of atoms due to PTM.",
   "mod_loss": "Loss of atoms due to PTM.",
   "red_smiles": "Reduced SMILES representation of PTM."
}.
When writing your own feature extractor, you can either
    (1) use the FeatureExtractor class or
    (2) write a function that can be mapped to the Hugging Face dataset.
In both cases, you can access the parsed sequence information from the dataset using the following keys, which all provide python lists:
    - _parsed_sequence: parsed sequence
    - _n_term_mods: N-terminal modifications
    - _c_term_mods: C-terminal modifications



2024-07-19 07:45:31.373179: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-07-19 07:45:31.373235: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: dragon.exbio.wzw.tum.de
2024-07-19 07:45:31.373244: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: dragon.exbio.wzw.tum.de
2024-07-19 07:45:31.373352: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 535.129.3
2024-07-19 07:45:31.373384: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 535.129.3
2024-07-19 07:45:31.373391: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:241] kernel version seems to match DSO: 535.129.3


Model: "prosit_intensity_predictor"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  432       
                                                                 
 sequential (Sequential)     (None, 30, 512)           1996800   
                                                                 
 sequential_1 (Sequential)   multiple                  4608      
                                                                 
 sequential_2 (Sequential)   (None, 29, 512)           1576806   
                                                                 
 encoder_att (AttentionLaye  multiple                  542       
 r)                                                              
                                                                 
 sequential_3 (Sequential)   multiple                  0         
                                        

In [4]:
dense_weights = model.regressor.get_layer("time_dense").get_weights()[0]
dense_bias = model.regressor.get_layer("time_dense").get_weights()[1]

In [5]:
model.len_fion = 3 * 4
model.regressor = tf.keras.Sequential(
        [
            tf.keras.layers.TimeDistributed(
                tf.keras.layers.Dense(model.len_fion), name='time_dense'
                ), 
            tf.keras.layers.LeakyReLU(name='activation'), 
            tf.keras.layers.Flatten(name='out')], 
        name='regressor'
        )


In [6]:
model.compile(tf.keras.optimizers.Adam(learning_rate=0.0001), loss=masked_spectral_distance)

In [12]:
ds = load_processed_dataset('/cmnfs/proj/bmpc_dlomix/datasets/processed/noptm_baseline_small_duplicated_ions')

In [9]:
batch, _ = next(iter(ds.tensor_train_data))
model(batch)
model.summary()

Model: "prosit_intensity_predictor"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  432       
                                                                 
 sequential (Sequential)     (None, 30, 512)           1996800   
                                                                 
 sequential_1 (Sequential)   multiple                  4608      
                                                                 
 sequential_2 (Sequential)   (None, 29, 512)           1576806   
                                                                 
 encoder_att (AttentionLaye  multiple                  542       
 r)                                                              
                                                                 
 sequential_3 (Sequential)   multiple                  0         
                                        

In [13]:
model.regressor.get_layer("time_dense").layer.set_weights([K.concatenate([dense_weights, dense_weights], axis=1), K.concatenate([dense_bias, dense_bias], axis=0)])

In [14]:
model.summary()

Model: "prosit_intensity_predictor"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  432       
                                                                 
 sequential (Sequential)     (None, 30, 512)           1996800   
                                                                 
 sequential_1 (Sequential)   multiple                  4608      
                                                                 
 sequential_2 (Sequential)   (None, 29, 512)           1576806   
                                                                 
 encoder_att (AttentionLaye  multiple                  542       
 r)                                                              
                                                                 
 sequential_3 (Sequential)   multiple                  0         
                                        

In [15]:
model.save("/cmnfs/proj/bmpc_dlomix/models/refinement_transfer_learning/double_regressor1.keras")

In [16]:
new_model = tf.keras.models.load_model("/cmnfs/proj/bmpc_dlomix/models/refinement_transfer_learning/double_regressor1.keras")
new_model.summary()

ValueError: Cannot assign value to variable ' time_dense/kernel:0': Shape mismatch.The variable shape (512, 6), and the assigned value shape (512, 12) are incompatible.